In [1]:
import pandas as pd
import numpy as np
import math as m
from scipy.optimize import fsolve

In [2]:
# Original DataFrame
unitWeights = {
    'Depth[m]': [0, 6, 6.01, 12, 12.01, 18, 18.01, 24, 24.01, 30, 30.01, 60],
    'effUnitWeight[kN/m3]': [7, 7, 8, 8, 8, 8, 9, 9, 10, 10, 9, 9],
}

df = pd.DataFrame(unitWeights)

# Create a new depth range with 0.01 m increments
new_depths = np.arange(0, df['Depth[m]'].max() + 0.01, 0.01)

# Interpolate the unit weights
unitWeights_df = pd.DataFrame({'Depth[m]': new_depths})
unitWeights_df['effUnitWeight[kN/m3]'] = np.interp(
    new_depths, 
    df['Depth[m]'], 
    df['effUnitWeight[kN/m3]']
)

# Display the resulting DataFrame
print(unitWeights_df)


      Depth[m]  effUnitWeight[kN/m3]
0         0.00                   7.0
1         0.01                   7.0
2         0.02                   7.0
3         0.03                   7.0
4         0.04                   7.0
...        ...                   ...
5996     59.96                   9.0
5997     59.97                   9.0
5998     59.98                   9.0
5999     59.99                   9.0
6000     60.00                   9.0

[6001 rows x 2 columns]


In [3]:
# Create a deep copy
effSigma_df = unitWeights_df.copy(deep=True)


# Add a new column for vertical effective stress
effSigma_df['VerticalEffectiveStress[kPa]'] = 0.0

# Compute the vertical effective stress iteratively
for i in range(1, len(effSigma_df)):
    # Get the depth difference
    depth_diff = effSigma_df.loc[i, 'Depth[m]'] - effSigma_df.loc[i - 1, 'Depth[m]']
    # Compute the effective stress
    effSigma_df.loc[i, 'VerticalEffectiveStress[kPa]'] = (
        effSigma_df.loc[i - 1, 'VerticalEffectiveStress[kPa]'] +
        effSigma_df.loc[i, 'effUnitWeight[kN/m3]'] * depth_diff
    )

# Display the resulting DataFrame
print(effSigma_df.to_string())


      Depth[m]  effUnitWeight[kN/m3]  VerticalEffectiveStress[kPa]
0         0.00                   7.0                          0.00
1         0.01                   7.0                          0.07
2         0.02                   7.0                          0.14
3         0.03                   7.0                          0.21
4         0.04                   7.0                          0.28
5         0.05                   7.0                          0.35
6         0.06                   7.0                          0.42
7         0.07                   7.0                          0.49
8         0.08                   7.0                          0.56
9         0.09                   7.0                          0.63
10        0.10                   7.0                          0.70
11        0.11                   7.0                          0.77
12        0.12                   7.0                          0.84
13        0.13                   7.0                          

In [4]:
def compute_f_integral(target_depth, func, *args, **kwargs):
    df = func(*args, **kwargs)  # Apply the transformation function
    if 'Pu' not in df.columns or 'EquivalentDepth[m]' not in df.columns:
        raise ValueError("The input function must output a DataFrame with 'Pu' and 'EquivalentDepth[m]' columns.")
    df['F-integral'] = 0.0
    for i in range(1, len(df)):
        Pu_current = df.loc[i, 'Pu']
        Pu_previous = df.loc[i - 1, 'Pu']
        Depth_current = df.loc[i, 'EquivalentDepth[m]']
        Depth_previous = df.loc[i - 1, 'EquivalentDepth[m]']
        delta_F = 0.5 * (Pu_current + Pu_previous) * (Depth_current - Depth_previous)
        df.loc[i, 'F-integral'] = df.loc[i - 1, 'F-integral'] + delta_F
    f_integral_at_target = df.loc[df['Depth[m]'] == target_depth, 'F-integral'].values
    return f_integral_at_target[0] if len(f_integral_at_target) > 0 else None

def api_clay_pu(df, Su, D, J, d_adj=0):
    df = df.copy(deep=True)
    df['EquivalentDepth[m]'] = np.maximum(df['Depth[m]'] + d_adj, 0)
    df['Pu_1'] = (3 * Su + df['VerticalEffectiveStress[kPa]']) * D + J * Su * df['EquivalentDepth[m]']
    df['Pu_2'] = 9 * Su * D
    df['Pu'] = np.minimum(df['Pu_1'], df['Pu_2'] )
    return df

def api_sand_pu(df, phi, D, d_adj=0):
    df = df.copy(deep=True)
    # initial subgrade modulus in kpa from fit with API (2014)

    # Calculate Pmax (regular API)
    ## Factors according to Mosher and Dawkins 2008.(regular API)
    b = 0.4
    Beta = 45 + phi / 2
    rad = m.pi / 180
    C1 = (
        (b * m.tan(phi * rad) * m.sin(Beta * rad))
        / (m.tan((Beta - phi) * rad) * m.cos((phi / 2) * rad))
        + ((m.tan(Beta * rad)) ** 2 * m.tan((phi / 2) * rad)) / (m.tan((Beta - phi) * rad))
        + b * m.tan(Beta * rad) * (m.tan(phi * rad) * m.sin(Beta * rad) - m.tan((phi / 2) * rad))
    )
    C2 = m.tan(Beta * rad) / m.tan((Beta - phi) * rad) - (m.tan((45 - phi / 2) * rad)) ** 2
    C3 = b * m.tan(phi * rad) * (m.tan(Beta * rad)) ** 4 + (m.tan((45 - phi / 2) * rad)) ** 2 * (
        (m.tan(Beta * rad)) ** 8 - 1
    )

    df['EquivalentDepth[m]'] = np.maximum(df['Depth[m]'] + d_adj, 0)

    df['Pu_s'] = (C1 * df['VerticalEffectiveStress[kPa]'] * df['EquivalentDepth[m]']) + C2 * df['VerticalEffectiveStress[kPa]'] * D
    df['Pu_d'] = C3 * df['VerticalEffectiveStress[kPa]'] * D

    ## Pmax for shallow and deep zones (regular API)
    df['Pu'] = np.minimum(df['Pu_s'], df['Pu_d'])
    return df

def frankeRollins2013_pu(df, Sr, D, J, d_adj=0):
    if D > 2.6:
        p_d = 9.24
    elif D > 0.3:
        p_d = 3.81 * np.log(D) + 5.6
    else:
        p_d = (D / 0.3) * 3.81 * np.log(0.3) + 5.6

    df['EquivalentDepth[m]'] = df['Depth[m]']
    df['z_depth'] = np.minimum(6.0, df['VerticalEffectiveStress[kPa]']/10.0)

    # derive static curve Rollins 2005
    df['A'] = (0.0000003) * ((df['z_depth'] + 1) ** 6.05)
    df['B'] = 2.80 * ((df['z_depth'] + 1) ** 0.11)
    df['C'] = 2.85 * ((df['z_depth'] + 1) ** -0.41)
    
    df['Pu_Rollins']  = np.minimum((df['A'] * ((df['B'] *  150)) ** df['C']), 15) * (p_d)    
    df['Pu1'] = (3 + (df['VerticalEffectiveStress[kPa]']/Sr) + (J/D)*df['Depth[m]']) * Sr * D
    df['Pu2'] = 9 * Sr * D
    df['Pu_wangReese'] = np.minimum(df['Pu1'], df['Pu2'])
    df['Pu'] = np.minimum(df['Pu_wangReese'], df['Pu_Rollins'])
    return df


# Layer 1

In [ ]:
phi = 31.0  # phi = 35.0  # Friction Angle
D = 2.0    # Diameter
target_depth = 6.0  # Target depth (m)
Sigma_df = effSigma_df.copy(deep=True)

F1 = compute_f_integral(target_depth, api_sand_pu, Sigma_df, phi, D, d_adj=0)

print(f"F-integral is {F1} at {target_depth} m ")

F-integral is 1759.2779460735846 at 6.0 m 


# Layer 2

In [7]:
phi = 35.0  # Friction Angle
D = 2.0    # Diameter
target_depth = 6.0  # Target depth (m)
target_value = F1  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

solution = fsolve(lambda x: compute_f_integral(target_depth, api_sand_pu, Sigma_df, phi, D, d_adj = x) - target_value, 
    x0=0.0,)

print(f"d_adj found: {solution[0]}")

F0 = compute_f_integral(
    target_depth,
    api_sand_pu,
    effSigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)

next_depth = 12.0

F1 = compute_f_integral(
    next_depth,
    api_sand_pu,
    effSigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)


print(f"F-integral at {target_depth} m with d_adj = {solution[0]}: {F0}")
print(f"F-integral at {next_depth} m with d_adj = {solution[0]}: {F1}")

d_adj found: -1.490280463302738
F-integral at 6.0 m with d_adj = -1.490280463302738: 1759.277946073585
F-integral at 12.0 m with d_adj = -1.490280463302738: 13728.679321304287


# Layer 3

In [8]:
phi = 36.0  # Friction Angle
D = 2.0    # Diameter
target_depth = 12.0  # Target depth (m)
target_value = F1  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

solution = fsolve(lambda x: compute_f_integral(target_depth, api_sand_pu, Sigma_df, phi, D, d_adj = x) - target_value, 
    x0=0.0,)

print(f"d_adj found: {solution[0]}")

F0 = compute_f_integral(
    target_depth,
    api_sand_pu,
    effSigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)

next_depth = 18.0

F1 = compute_f_integral(
    next_depth,
    api_sand_pu,
    effSigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)


print(f"F-integral at {target_depth} m with d_adj = {solution[0]}: {F0}")
print(f"F-integral at {next_depth} m with d_adj = {solution[0]}: {F1}")

d_adj found: -2.130122267621226
F-integral at 12.0 m with d_adj = -2.130122267621226: 13728.6793213043
F-integral at 18.0 m with d_adj = -2.130122267621226: 47664.78025884199


# Layer 4

In [9]:
phi = 37.0  # Friction Angle
D = 2.0    # Diameter
target_depth = 18.0  # Target depth (m)
target_value = F1  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

solution = fsolve(lambda x: compute_f_integral(target_depth, api_sand_pu, Sigma_df, phi, D, d_adj = x) - target_value, 
    x0=0.0,)

print(f"d_adj found: {solution[0]}")

F0 = compute_f_integral(
    target_depth,
    api_sand_pu,
    effSigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)

next_depth = 24.0

F1 = compute_f_integral(
    next_depth,
    api_sand_pu,
    effSigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)


print(f"F-integral at {target_depth} m with d_adj = {solution[0]}: {F0}")
print(f"F-integral at {next_depth} m with d_adj = {solution[0]}: {F1}")

d_adj found: -3.0639344827240724
F-integral at 18.0 m with d_adj = -3.0639344827240724: 47664.78025884197
F-integral at 24.0 m with d_adj = -3.0639344827240724: 118620.05281904592


# Layer 5

In [13]:
Su = 192
D = 2.0    # Diameter
J = 0.5    # Factor
target_depth = 24.0  # Target depth (m)
Sigma_df = effSigma_df.copy(deep=True)
F2 = compute_f_integral(target_depth, api_clay_pu, Sigma_df, Su, D, J, d_adj=0)

target_depth = 30.0

F3 = compute_f_integral(target_depth, api_clay_pu, Sigma_df, Su, D, J, d_adj=0) + (F1-F2)

print(f"F-integral is {F3} at {target_depth} m ")

F-integral is 124416.0000000014 at 30.0 m 


# Layer 6

In [14]:
phi = 39.0  # Friction Angle
D = 2.0    # Diameter
target_depth = 30.0 # Target depth (m)
target_value = F3  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

solution = fsolve(lambda x: compute_f_integral(target_depth, api_sand_pu, Sigma_df, phi, D, d_adj = x) - target_value, 
    x0=0.0,)

print(f"d_adj found: {solution[0]}")

F0 = compute_f_integral(
    target_depth,
    api_sand_pu,
    effSigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)

next_depth = 60.0

F1 = compute_f_integral(
    next_depth,
    api_sand_pu,
    effSigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)


print(f"F-integral at {target_depth} m with d_adj = {solution[0]}: {F0}")
print(f"F-integral at {next_depth} m with d_adj = {solution[0]}: {F1}")

d_adj found: -14.577773130626385
F-integral at 30.0 m with d_adj = -14.577773130626385: 124416.00000000138
F-integral at 60.0 m with d_adj = -14.577773130626385: 1780040.591027471


In [6]:
phi = 35.0  # Friction Angle
D = 2.0    # Diameter
target_depth = 4.0  # Target depth (m)
Sigma_df = effSigma_df.copy(deep=True)
F2 = compute_f_integral(target_depth, api_sand_pu, Sigma_df, phi, D, d_adj=0)

target_depth = 10.5

F3 = compute_f_integral(target_depth, frankeRollins2013_pu, Sigma_df, Sr, D, J, d_adj=0) + (F1-F2)

print(f"F-integral is {F3} at {target_depth} m ")


F-integral is 1160.4331984438058 at 10.5 m 


# Layer 3

In [7]:
Sr = 15.6102761856926
D = 2.0    # Diameter
J = 0.5    # Factor
target_depth = 10.5  # Target depth (m)
Sigma_df = effSigma_df.copy(deep=True)
F4 = compute_f_integral(target_depth, frankeRollins2013_pu, Sigma_df, Sr, D, J, d_adj=0)

target_depth = 12.0

F5 = compute_f_integral(target_depth, frankeRollins2013_pu, Sigma_df, Sr, D, J, d_adj=0) + (F3-F4)

print(f"F-integral is {F5} at {target_depth} m ")


F-integral is 1345.8532404973207 at 12.0 m 


# Layer 4

In [8]:
Sr = 30.0137074898023
D = 2.0    # Diameter
J = 0.5    # Factor
target_depth = 12.0  # Target depth (m)
Sigma_df = effSigma_df.copy(deep=True)
F6 = compute_f_integral(target_depth, frankeRollins2013_pu, Sigma_df, Sr, D, J, d_adj=0)

target_depth = 19.5

F7 = compute_f_integral(target_depth, frankeRollins2013_pu, Sigma_df, Sr, D, J, d_adj=0) + (F5-F6)

print(f"F-integral is {F7} at {target_depth} m ")


F-integral is 2272.95345076487 at 19.5 m 


# Layer 5

In [9]:
Su = 70.0  # Shear strength
D = 2.0    # Diameter
J = 0.5    # Factor
target_depth = 19.5  # Target depth (m)
target_value = F7  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

solution = fsolve(lambda x: compute_f_integral(target_depth, api_clay_pu, Sigma_df, Su, D, J, d_adj = x) - target_value, 
    x0=0.0,)

print(f"d_adj found: {solution[0]}")

F0 = compute_f_integral(
    target_depth,
    api_clay_pu,
    effSigma_df,
    Su=Su,
    D=D,
    J=J,
    d_adj=solution[0]
)

next_depth = 36.0

F1 = compute_f_integral(
    next_depth,
    api_clay_pu,
    effSigma_df,
    Su=Su,
    D=D,
    J=J,
    d_adj=solution[0]
)


print(f"F-integral at {target_depth} m with d_adj = {solution[0]}: {F0}")
print(f"F-integral at {next_depth} m with d_adj = {solution[0]}: {F1}")

d_adj found: -15.820163244982751
F-integral at 19.5 m with d_adj = -15.820163244982751: 2272.9534507648673
F-integral at 36.0 m with d_adj = -15.820163244982751: 19826.211273227203


# Layer 6

In [10]:
phi = 37.0
D = 2.0
target_depth = 36.0  # Target depth (m)
target_value = F1  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

solution = fsolve(lambda x: compute_f_integral(target_depth, api_sand_pu, Sigma_df, phi, D, d_adj = x) - target_value, 
    x0=0.0,)

print(f"d_adj found: {solution[0]}")

F0 = compute_f_integral(
    target_depth,
    api_sand_pu,
    Sigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)

next_depth = 52.5

F1 = compute_f_integral(
    next_depth,
    api_sand_pu,
    Sigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)


print(f"F-integral at {target_depth} m with d_adj = {solution[0]}: {F0}")
print(f"F-integral at {next_depth} m with d_adj = {solution[0]}: {F1}")

d_adj found: -29.788589323442405
F-integral at 36.0 m with d_adj = -29.788589323442405: 19826.2112732272
F-integral at 52.5 m with d_adj = -29.788589323442405: 278247.3287178255


# Layer 7


In [11]:
phi = 40
D = 2.0
target_depth = 52.5  # Target depth (m)
target_value = F1  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

print(f"target F1 = {target_value}")

solution = fsolve(lambda x: compute_f_integral(target_depth, api_sand_pu, Sigma_df, phi, D, d_adj = x) - target_value, 
    x0=0.0,)

print(f"d_adj found: {solution[0]}")

F0 = compute_f_integral(
    target_depth,
    api_sand_pu,
    Sigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)

next_depth = 60.0

F1 = compute_f_integral(
    next_depth,
    api_sand_pu,
    Sigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)


print(f"F-integral at {target_depth} m with d_adj = {solution[0]}: {F0}")
print(f"F-integral at {next_depth} m with d_adj = {solution[0]}: {F1}")

target F1 = 278247.3287178255
d_adj found: -33.014800015762
F-integral at 52.5 m with d_adj = -33.014800015762: 278247.32871782564
F-integral at 60.0 m with d_adj = -33.014800015762: 592745.4995407446
